# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [159]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def rename_team(x):
    x = x.split()[-1]
    if x == 'Wings':
        x = 'Red Wings'
    if x == 'Leafs':
        x = 'Maple Leafs'
    if x == 'Jackets':
        x = 'Blue Jackets'
    if x == 'Knights':
        x = 'Golden Knights'
    return x

def win_lose_ratio(x):
    x['W/L'] = float(x['W'])/ (float(x['W']) + float(x['L']))
    return x


def cut_names(x):
    if re.findall('\[note [0-9]+\]', x):
        x = re.sub(r'\[note [0-9]+\]', '', x)
    if len(x) == 0:
        x = np.NaN
    elif x == '—':
        x = np.NaN
    return x


def clear_wildcard(x):
    x[0] = re.sub(r'\*', '', x[0])
    return x


def nhl_correlation():
    nhl_df = pd.read_csv("assets/nhl.csv")
    nhl_df = nhl_df[nhl_df['year'] == 2018]
    nhl_df = nhl_df.apply(clear_wildcard, axis=1)
    nhl_df = nhl_df.set_index('team')
    nhl_df = nhl_df.drop(index=['Atlantic Division', 'Metropolitan Division', 'Central Division', 'Pacific Division'])
    nhl_df = nhl_df.apply(win_lose_ratio, axis=1)
    nhl_df = nhl_df[['W/L']]
    nhl_df = nhl_df.reset_index()
    nhl_df['team'] = nhl_df['team'].apply(rename_team)
    nhl_df = nhl_df.set_index('team')
    
    cities = pd.read_html("assets/wikipedia_data.html")[1]
    cities = cities.iloc[:-1,[0,3,5,6,7,8]]
    cities['NHL'] = cities['NHL'].apply(cut_names)
    cities = cities[['NHL', 'Metropolitan area', 'Population (2016 est.)[8]']].set_index('Metropolitan area')
    cities = cities[cities['NHL'].notna()]
    cities = cities.reset_index()
    cities = cities.set_index('NHL')
    combined = pd.merge(cities, nhl_df, how='left', right_index=True, left_index=True)
    combined = combined.reset_index()
    combined = combined.set_index('Metropolitan area')
    wl_combined = combined[['W/L']]
    wl_combined.loc['New York City']['W/L'] = (nhl_df.loc['Rangers']['W/L'] + nhl_df.loc['Islanders']['W/L'] + nhl_df.loc['Devils']['W/L']) / 3
    wl_combined.loc['Los Angeles']['W/L'] = (nhl_df.loc['Kings']['W/L'] + nhl_df.loc['Ducks']['W/L']) / 2
    combined['W/L'] = wl_combined['W/L']
    
    combined['Population (2016 est.)[8]'] = combined['Population (2016 est.)[8]'].apply(lambda x: np.int(x))
    combined = combined.reset_index()

    population_by_region = combined['Population (2016 est.)[8]']  # pass in metropolitan area population from cities
    win_loss_by_region = combined['W/L']                          # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    assert len(population_by_region) == len(win_loss_by_region),  "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28,                       "Q1: There should be 28 teams being analysed for NHL"
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]


nhl_correlation()


0.012486162921209912

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [177]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def rename_team(x):
    x = x.split()[-1]
    if x == 'Blazers':
        x = 'Trail Blazers'
    return x


def win_lose_ratio(x):
    x['W/L'] = float(x['W'])/ (float(x['W']) + float(x['L']))
    return x

def cut_names(x):
    if re.findall('\[note [0-9]+\]', x):
        x = re.sub(r'\[note [0-9]+\]', '', x)
    if len(x) == 0:
        x = np.NaN
    elif x == '—':
        x = np.NaN
    return x


def nba_correlation():
    nba_df=pd.read_csv("assets/nba.csv")
    nba_df = nba_df[nba_df['year'] == 2018]
    nba_df['team'] = nba_df['team'].apply(lambda x: re.sub(r'\*', '', x))
    nba_df['team'] = nba_df['team'].apply(lambda x: re.sub(r'\([0-9]+\)', '', x))
    nba_df = nba_df.apply(win_lose_ratio, axis=1)
    nba_df = nba_df.set_index('team')
    nba_df = nba_df[['W/L']]
    nba_df = nba_df.reset_index()
    nba_df['team'] = nba_df['team'].apply(rename_team)
    nba_df = nba_df.set_index('team')

    cities = pd.read_html("assets/wikipedia_data.html")[1]
    cities = cities.iloc[:-1,[0,3,5,6,7,8]]
    cities['NBA'] = cities['NBA'].apply(cut_names)
    cities = cities[['NBA', 'Metropolitan area', 'Population (2016 est.)[8]']].set_index('Metropolitan area')
    cities = cities[cities['NBA'].notna()]
    cities = cities.reset_index()
    cities = cities.set_index('NBA')
    
    combined = pd.merge(cities, nba_df, how='left', right_index=True, left_index=True)
    combined = combined.reset_index()
    combined = combined.set_index('Metropolitan area')
    wl_combined = combined[['W/L']]
    wl_combined.loc['New York City']['W/L'] = (nba_df.loc['Knicks']['W/L'] + nba_df.loc['Nets']['W/L']) / 2
    wl_combined.loc['Los Angeles']['W/L'] = (nba_df.loc['Lakers']['W/L'] + nba_df.loc['Clippers']['W/L']) / 2
    combined['W/L'] = wl_combined['W/L']
    combined['Population (2016 est.)[8]'] = combined['Population (2016 est.)[8]'].apply(lambda x: np.int(x))
    combined = combined.reset_index()
    
    population_by_region = combined['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = combined['W/L']                         # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28,                      "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

                          
nba_correlation()

-0.1765716025284462

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [186]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def rename_team(x):
    if x == 'Boston Red Sox':
        return 'Red Sox'
    x = x.split()[-1]
    if x == 'Jays':
        x = 'Blue Jays'
    return x


def win_lose_ratio(x):
    x['W/L'] = float(x['W'])/ (float(x['W']) + float(x['L']))
    return x


def cut_names(x):
    if re.findall('\[note [0-9]+\]', x):
        x = re.sub(r'\[note [0-9]+\]', '', x)
    if len(x) == 0:
        x = np.NaN
    elif x == '—':
        x = np.NaN
    return x


def mlb_correlation():
    mlb_df=pd.read_csv("assets/mlb.csv")
    mlb_df = mlb_df[mlb_df['year'] == 2018]
    mlb_df = mlb_df.apply(win_lose_ratio, axis=1)
    mlb_df = mlb_df.set_index('team')
    mlb_df = mlb_df[['W/L']]
    mlb_df = mlb_df.reset_index()
    mlb_df['team'] = mlb_df['team'].apply(rename_team)
    mlb_df = mlb_df.set_index('team')

    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities['MLB'] = cities['MLB'].apply(cut_names)
    cities = cities[['MLB', 'Metropolitan area', 'Population (2016 est.)[8]']].set_index('Metropolitan area')
    cities = cities[cities['MLB'].notna()]
    cities = cities.reset_index()
    cities = cities.set_index('MLB')

    combined = pd.merge(cities, mlb_df, how='left', right_index=True, left_index=True)
    combined = combined.reset_index()
    combined = combined.set_index('Metropolitan area')
    wl_combined = combined[['W/L']]
    wl_combined.loc['New York City']['W/L'] = (mlb_df.loc['Yankees']['W/L'] + mlb_df.loc['Mets']['W/L']) / 2
    wl_combined.loc['Los Angeles']['W/L'] = (mlb_df.loc['Dodgers']['W/L'] + mlb_df.loc['Angels']['W/L']) / 2
    wl_combined.loc['San Francisco Bay Area']['W/L'] = (mlb_df.loc['Giants']['W/L'] + mlb_df.loc['Athletics']['W/L']) / 2
    wl_combined.loc['Chicago']['W/L'] = (mlb_df.loc['Cubs']['W/L'] + mlb_df.loc['Sox']['W/L']) / 2
    combined['W/L'] = wl_combined['W/L']
    combined['Population (2016 est.)[8]'] = combined['Population (2016 est.)[8]'].apply(lambda x: np.int(x))
    combined = combined.reset_index()
    
    population_by_region = combined['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = combined['W/L']                         # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26,                      "Q3: There should be 26 teams being analysed for MLB"
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]


mlb_correlation()


0.1502769830266931

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [219]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


def rename_team(x):
    x = re.sub(r'[\+\*]', '', x)
    x = x.split()[-1]
    return x


def win_lose_ratio(x):
    x['W/L'] = float(x['W'])/ (float(x['W']) + float(x['L']))
    return x


def cut_names(x):
    if re.findall('\[note [0-9]+\]', x):
        x = re.sub(r'\[note [0-9]+\]', '', x)
    if len(x) == 0:
        x = np.NaN
    elif x == '—' or x == '— ':
        x = np.NaN
    return x


def nfl_correlation(): 
    nfl_df = pd.read_csv("assets/nfl.csv")
    nfl_df = nfl_df[nfl_df['year'] == 2018]
    nfl_df = nfl_df.set_index('team')
    nfl_df = nfl_df.drop(index=['AFC East', 'AFC North', 'AFC South', 'AFC West', 'NFC East', 'NFC North', 'NFC South', 'NFC West'])
    nfl_df = nfl_df.reset_index()
    nfl_df = nfl_df.apply(win_lose_ratio, axis=1)
    nfl_df = nfl_df.set_index('team')
    nfl_df = nfl_df[['W/L']]
    nfl_df = nfl_df.reset_index()
    nfl_df['team'] = nfl_df['team'].apply(rename_team)
    nfl_df = nfl_df.set_index('team')

    cities = pd.read_html("assets/wikipedia_data.html")[1]
    cities = cities.iloc[:-1,[0,3,5,6,7,8]]
    cities['NFL'] = cities['NFL'].apply(cut_names)
    cities = cities[['NFL', 'Metropolitan area', 'Population (2016 est.)[8]']].set_index('Metropolitan area')
    cities = cities[cities['NFL'].notna()]
    cities = cities.reset_index()
    cities = cities.set_index('NFL')

    combined = pd.merge(cities, nfl_df, how='left', right_index=True, left_index=True)
    combined = combined.reset_index()
    combined = combined.set_index('Metropolitan area')
    wl_combined = combined[['W/L']]
    wl_combined.loc['New York City']['W/L'] = (nfl_df.loc['Giants']['W/L'] + nfl_df.loc['Jets']['W/L']) / 2
    wl_combined.loc['Los Angeles']['W/L'] = (nfl_df.loc['Rams']['W/L'] + nfl_df.loc['Chargers']['W/L']) / 2
    wl_combined.loc['San Francisco Bay Area']['W/L'] = (nfl_df.loc['49ers']['W/L'] + nfl_df.loc['Raiders']['W/L']) / 2
    combined['W/L'] = wl_combined['W/L']
    combined['Population (2016 est.)[8]'] = combined['Population (2016 est.)[8]'].apply(lambda x: np.int(x))
    combined = combined.reset_index()

    population_by_region = combined['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = combined['W/L']                         # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29,                      "Q4: There should be 29 teams being analysed for NFL"
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]


nfl_correlation()

0.004922112149349386

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [3]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


def sports_team_performance():
        None
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
#     sports = ['NFL', 'NBA', 'NHL', 'MLB']
#     p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
#     assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
#     assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
#     return p_values


sports_team_performance()